# BQ Physical Storage

전체 Table의 Logical / Physical Storage 정보를 가져오고, 압축 효과가 가장 높은 Table들 Sort해서 보여줌

In [ ]:
!pip3 install pandas python-dotenv

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

In [9]:
PROJECT_ID = os.environ['PROJECT_ID']

In [12]:
TOKEN = ! gcloud auth print-access-token

In [14]:
import google.auth.transport.requests
import requests

def get_table(project, dataset, table):
    url = f"https://bigquery.googleapis.com/bigquery/v2/projects/{project}/datasets/{dataset}/tables/{table}"

    try:
    
        headers = {"Authorization": f"Bearer {TOKEN[0]}"}
        response = requests.get(url, headers=headers)

        # Raise an exception for any HTTP error status codes
        response.raise_for_status()

        # Parse the JSON response
        json_output = response.json()
        return json_output

    except Exception as e:
        print(f"Error making API request: {e}")
        return None

In [ ]:
from google.cloud import bigquery
import pandas as pd

def get_bigquery_storage_info(project_id):

    df = pd.DataFrame(columns=["dataset", "table", "logical", "physical", "ratio"])
    df_need_to_check = pd.DataFrame(columns=["dataset", "table", "logical", "physical", "ratio"])


    client = bigquery.Client(project=project_id)

    # Get list of datasets
    datasets = list(client.list_datasets())
    if not datasets:
        print(f"No datasets found in project {project_id}")
        return

    for dataset in datasets:
        dataset_id = dataset.dataset_id
        #location = dataset.location
        print(f"Dataset: {dataset_id} ")

        # Get list of tables in the dataset
        tables = list(client.list_tables(dataset_id))
        if not tables:
            print(f"  No tables found in dataset {dataset_id}")
            continue

        for table in tables:
            table_id = table.table_id
            table_json = get_table(project_id, dataset_id, table_id)
            #print(table_json)

            if "numTotalLogicalBytes" in table_json:
                logical = int(table_json['numTotalLogicalBytes'])
            else:
                logical = 0

            if "numTotalPhysicalBytes" in table_json:
                physical = int(table_json['numTotalPhysicalBytes'])
            else:
                physical = 0
                     
            if physical == 0:
                ratio = 0
            else:
                ratio = logical / physical 

            #print(f"{dataset_id} {table_id} {logical}   {physical}  {ratio}")
            if physical > logical:
                df_need_to_check = df_need_to_check._append({'dataset': dataset_id, 'table': table_id,'logical':logical ,'physical':physical,'ratio':ratio }, ignore_index=True)     
            else:
                df = df._append({'dataset': dataset_id, 'table': table_id,'logical':logical ,'physical':physical,'ratio':ratio }, ignore_index=True)
    
    return df, df_need_to_check


In [ ]:
tables, tables_to_check = get_bigquery_storage_info(PROJECT_ID)

In [17]:
display(tables.head())

,dataset,table,logical,physical,ratio
0,billing_export,gcp_billing_export_resource_v1_01F6D3_EB24B5_C...,41595797,5791390,7.182351
1,billing_export,gcp_billing_export_v1_01F6D3_EB24B5_C93EEB,1316506873,48367413,27.218881
2,codelab,cta_ridership,111024,32840,3.380755
3,dataform,first_view,0,0,0.000000
4,dataform,quickstart-source,0,0,0.000000


### Total Logical Storage, Total Physical Storage

In [18]:
total_logical= tables['logical'].sum()
total_physical= tables['physical'].sum()
print(f"Total Logical : {total_logical/1024/1024/1024:.2f} GB")
print(f"Total physical : {total_physical/1024/1024/1024:.2f} GB")

Total Logical : 65.20 GB
Total physical : 7.12 GB


### 가장 압축율이 좋은 순으로 정렬 

In [22]:
table_sorted = tables.sort_values(by='ratio', ascending=False)
display(table_sorted.head(5))

,dataset,table,logical,physical,ratio
49,tsop,tsop_job_logs,23797696,41709,570.565010
50,tsop,tsop_log_logs_2,23797696,41713,570.510297
48,tsop,temp_op,2964624,7268,407.900936
14,ecommerce,traning_data,52591396,978024,53.773114
13,ecommerce,train-data,52591396,978623,53.740200


### Physical Storage 용량이 Logical Storage 보다 큰 Table List

In [24]:
display(tables_to_check.head(4))

,dataset,table,logical,physical,ratio
0,dataform,quickstart-table,65,1194,0.054439
1,datastream,employees_departments,585,2919,0.200411
2,datastream,employees_dept_manager,1824,3463,0.526711
3,demo_demandforecasting,README,318,1544,0.205959
